In [1]:
import numpy as np
import pandas as pd

## 12.1　分类数据
本节介绍pandas的`Categorical`类型。将展示在使用pandas进行某些操作时如何获得更好的性能和内存使用。还会介绍一些在统计和机器学习应用中使用分类数据的工具。

### 12.1.1　背景和目标
一个列经常会包含重复值，这些重复值是一个小型的不同值的集合。已经看见像`unique`和`value_counts`这样的函数，它们允许从一个数组中提取不同值并分别计算这些不同值的频率：

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

许多数据系统（用于数据入库、统计计算或其他用途）已经开发出专门的方法，用重复的值来表示数据，以便更有效地存储和计算。在数据入库的操作中，使用所谓的维度表是一种最佳实践，维度表包含了不同值，并将主要观测值存储为引用维度表的整数键：

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(['apple', 'orange'])

print(values)
print('-'*50)
print(dim)

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64
--------------------------------------------------
0     apple
1    orange
dtype: object


可以使用`take`方法来恢复原来的字符串Series：

In [7]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种按照整数展现的方式被称为分类或字典编码展现。不同值的数组可以被称为数据的类别、字典或层级。这里将会使用分类的和类别这样的术语。

在做数据分析时，分类展示会产生显著的性能提升。也可以在类别上进行转换同时不改变代码。以下是一些相对低开销的转换示例：

* 重命名类别

* 在不改变已有的类别顺序的情况下添加一个新的类别

### 12.1.2　pandas中的Categorical类型
pandas拥有特殊的Categorical类型，用于承载基于整数的类别展示或编码的数据。考虑之前的示例Series：

In [12]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight']) # 这里的columns用来规整列的顺序

df

,basket_id,fruit,count,weight
0,0,apple,8,0.036074
1,1,orange,5,2.678417
2,2,apple,12,2.355048
3,3,apple,9,0.888169
4,4,apple,5,1.085836
5,5,orange,14,1.389083
6,6,apple,13,1.849245
7,7,apple,14,3.463552


这里，`df['fruit']`是一个Python字符串对象组成的数组。可以通过调用函数将它转换为Categorical对象：

In [13]:
fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

fruit_cat的值并不是NumPy数组，而是`pandas.Categorical`的实例：

In [19]:
c = fruit_cat.values

print(type(fruit_cat), type(c))
print('-'*50)
print(c)

<class 'pandas.core.series.Series'> <class 'pandas.core.arrays.categorical.Categorical'>
--------------------------------------------------
[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]


Categorical对象拥有`categories`和`codes`属性：

In [20]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [21]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

可以通过分配已转换的结果将DataFrame的一列转换为Categorical对象：

In [22]:
df['fruit'] = df['fruit'].astype('category')

df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

也可以从其他Python序列类型直接生成pandas.Categorical：

In [27]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

([foo, bar, baz, foo, bar]
 Categories (3, object): [bar, baz, foo],
 Index(['bar', 'baz', 'foo'], dtype='object'),
 array([2, 0, 1, 2, 0], dtype=int8))

如果已经从另一个数据源获得了分类编码数据，可以使用from_codes构造函数：

In [29]:
categories = ['foo', 'bar', 'baz']

codes = [0, 2, 1, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2

[foo, baz, bar, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

除非显式地指定，分类转换是不会指定类别的顺序的。因此categories数组可能会与输入数据的顺序不同。当使用`from_codes`或其他任意构造函数时，可以为类别指定一个有意义的顺序：

In [30]:
# 有顺序的分类组
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)

ordered_cat

[foo, baz, bar, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

输出的`[foo<bar<baz]`表明'foo'的顺序在'bar'之前，以此类推。一个未排序的分类实例可以使用`as_ordered`进行排序：

In [31]:
my_cats_2.as_ordered()

[foo, baz, bar, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

> 最后需要注意，分类数据可以不是字符串，尽管举的例子都是字符串例子。一个分类数组可以包含任一**不可变**的值类型。

### 12.1.3　使用Categorical对象进行计算
在pandas中使用Categorical与非编码版本相比（例如字符串数组）整体上是一致的。pandas中的某些部分，比如`groupby`函数，在与Categorical对象协同工作时性能更好。还有一些函数可以利用`ordered`标识。

考虑一些随机数字数据，并使用`pandas.qcut`分箱函数。结果会返回pandas.Categorical；之前使用过`pandas.cut`，但略过了分类如何工作的细节：

In [34]:
np.random.seed(12345)

draws = np.random.randn(1000)

draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

让计算上面数据的四分位分箱，并提取一些统计值：

In [35]:
bins = pd.qcut(draws, 4)

bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

虽然样本的四分位数有用，但是在生成一份报告时，四分位数就没有四分位数名称有用了。可以通过在`qcut`函数中使用`labels`参数来实现这个功能：

In [36]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [37]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

被标记的bins分类数据并不包含数据中箱体边界的相关信息，因此可以使用`groupby`来提取一些汇总统计值：

In [38]:
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


结果中的'quartile'列保留了bins中原始的分类信息，包括顺序：

In [39]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### 12.1.3.1　使用分类获得更高性能

如果在特定的数据集上做了大量的分析，将数据转换为分类数据可以产生大幅的性能提升。DateFrame中一列的分类版本通常也会明显使用更少内存。考虑一些含有一千万元素的Series以及少量的不同类别：

In [40]:
N = 10000000

draws = pd.Series(np.random.randn(N))

labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

现在将labels转换为Categorical对象：

In [41]:
categories = labels.astype('category')

现在可以发现到labels比categories使用了明显更多的内存：

In [42]:
print(labels.memory_usage())

print(categories.memory_usage())

80000080
10000272


分类转换当然不是免费的，但是它是一次性开销：

In [47]:
%time _ = labels.astype('category')

CPU times: user 858 ms, sys: 208 ms, total: 1.07 s
Wall time: 1.06 s


使用分类对象进行GroupBy操作明显更快，这是因为底层算法使用了基于整数代码的数组而不是字符串数组。

### 12.1.4　分类方法
Series包含的分类数据拥有一些特殊方法，这些方法类似于`Series.str`的特殊字符串方法。这些方法提供了快捷访问类别和代码的方式。考虑下面的Series：

In [48]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

特殊属性`cat`提供了对分类方法的访问：

In [49]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [50]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

假设知道该数据的实际类别集合超出了数据中观察到的四个值。可以使用`set_categories`方法来改变类别：

In [55]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

虽然看起来数据并未改变，但新类别将反映在使用它们的操作中。例如，`value_counts`将遵循新的类别（如果存在）：

In [57]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [58]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

在大型数据集中，分类数据经常被用于节省内存和更高性能的便捷工具。在过滤了一个大型DataFrame或Series之后，很多类别将不会出现在数据中。为了帮助解决这个问题，可以使用`remove_unused_categories`方法来去除未观察到的类别：

In [59]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [60]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

下表是pandas中Series的分类方法

| 方法 | 描述 |
| ---- | ---- |
| add_categories | 将新的类别(未使用过的)添加到已有类别的尾部 |
| as_ordered | 对类别排序 |
| as_unordered | 使类别无序 |
| remove_categories | 去除类别,将被移除的值置为null |
| remove\_unused\_categories | 去除所有没有出现在数据中的类别 |
| rename_categories | 使用新的类别名称替代现有的类别,不会改变类别的数量 |
| reorder_categories | 与 rename_categories类似,但结果是经过排序的类别 |
| set_categories | 用指定的一组新类别替换现有类别,可以添加或删除类别 |

#### 12.1.4.1　创建用于建模的虚拟变量

当你使用统计数据或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为`one-hot编码`。这会产生一个DataFrame，每个不同的类别都是它的一列。这些列包含一个特定类别的出现次数，否则为0。

考虑之前的一个例子：

In [61]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

之前在第7章中提到过，`pandas.get_dummies`函数将一维的分类数据转换为一个包含虚拟变量的DataFrame：

In [62]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2　高阶GroupBy应用
虽然已经在第10章深入讨论了在Series和DataFrame使用groupby方法，但仍然有一些额外的方法可能会用到。

### 12.2.1　分组转换和“展开”GroupBy
在第10章中，在分组操作中学习了`apply`方法用于执行转换操作。还有另一个内建方法`transform`，与`apply`方法类似但是会对可以使用的函数种类加上更多的限制：

* transform可以产生一个标量值，并广播到各分组的尺寸数据中

* transform可以产生一个与输入分组尺寸相同的对象

* transform不可改变它的输入

考虑一个简单的例子：

In [63]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


这里是按'key'分组的均值：

In [64]:
g = df.groupby('key').value

g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

假设想要产生一个Series，它的尺寸和df['value']一样，但值都被按'key'分组的均值替代。可以向`transfrom`传递匿名函数`lambda x：x.mean()`：

In [69]:
g.apply(lambda x: x.mean())

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [68]:
# 和apply函数放在一起比较就更清晰了
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

对于内建的聚合函数，可以像GroupBy的`agg`方法一样传递一个字符串别名：

In [70]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

与`apply`类似，`transform`可以与返回Series的函数一起使用，但结果必须和输入有相同的大小。例如，可以使用lambda函数给每个组乘以2：

In [75]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

作为一个更复杂的例子，可以按照每个组的降序计算排名：

In [76]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

考虑一个由简单聚合构成的分组转换函数：

In [77]:
def normalize(x):
    return (x - x.mean()) / x.std()

使用`transform`或`apply`可以获得等价的结果：

In [78]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [80]:
# 这里为什么不是分组后的结果，有点难以理解
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

内建的聚合函数如'mean'或'sum'通常会比`apply`函数更快。这些函数在与`transform`一起使用时也会存在一个"快速通过"。这允许执行一个所谓的展开分组操作：

In [81]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [82]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

然而一个展开分组操作可能会包含多个分组聚合，矢量化操作的整体优势往往超过了这一点。

### 12.2.2　分组的时间重新采样
对于时间序列数据，`resample`方法在语义上是一种基于时间分段的分组操作。下面是一个小的示例表：

In [87]:
N = 15

times = pd.date_range('2020-05-20 00:00', freq='1min', periods=N)

df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

df.head()

,time,value
0,2020-05-20 00:00:00,0
1,2020-05-20 00:01:00,1
2,2020-05-20 00:02:00,2
3,2020-05-20 00:03:00,3
4,2020-05-20 00:04:00,4


这里，可以按'time'进行索引，然后重新采样：

In [84]:
df.set_index('time').resample('5min').count()

,value
time,
2020-05-20 00:00:00,5
2020-05-20 00:05:00,5
2020-05-20 00:10:00,5


假设DataFrame包含多个时间序列，并按一个附加的分组键列进行了标记：

In [88]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})

df2.head()

,time,key,value
0,2020-05-20 00:00:00,a,0.0
1,2020-05-20 00:00:00,b,1.0
2,2020-05-20 00:00:00,c,2.0
3,2020-05-20 00:01:00,a,3.0
4,2020-05-20 00:01:00,b,4.0


要为每个'key'的值进行相同的重新采样，可以使用`pandas.Grouper`对象：

In [91]:
time_key = pd.Grouper(freq='5min')

之后可以设置时间索引，按'key'和time_key进行分组，再聚合：

In [92]:
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())

resampled

value
key time                      
a   2020-05-20 00:00:00   30.0
    2020-05-20 00:05:00  105.0
    2020-05-20 00:10:00  180.0
b   2020-05-20 00:00:00   35.0
    2020-05-20 00:05:00  110.0
    2020-05-20 00:10:00  185.0
c   2020-05-20 00:00:00   40.0
    2020-05-20 00:05:00  115.0
    2020-05-20 00:10:00  190.0

In [94]:
resampled.reset_index()

,key,time,value
0,a,2020-05-20 00:00:00,30.0
1,a,2020-05-20 00:05:00,105.0
2,a,2020-05-20 00:10:00,180.0
3,b,2020-05-20 00:00:00,35.0
4,b,2020-05-20 00:05:00,110.0
5,b,2020-05-20 00:10:00,185.0
6,c,2020-05-20 00:00:00,40.0
7,c,2020-05-20 00:05:00,115.0
8,c,2020-05-20 00:10:00,190.0


> 使用`Grouper`的一个限制是时间必须是Series或DataFrame的索引。

## 12.3　方法链技术
在向数据集应用一系列变换时，可能会发现自己创建了许多临时变量，而这些变量在分析中从未使用过。例如，考虑以下例子：

In [137]:
# df = load_data()
df = pd.DataFrame({'key' : np.random.randint(4, 9, 20),
                   'col1' : np.random.randint(20, 40, 20),
                   'col2' : np.random.randn(20)})

df2 = df[df['col2'] < 0]

In [135]:
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()

result = df2.groupby('key').col1_demeaned.std()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


尽管在这里并未使用真实数据，但是这个例子体现了一些新的方法。首先，`DataFrame.assign`方法是对df[k]=v的赋值方式的一种功能替代。它返回的是一个按指定修改的新的DataFrame，而不是在原对象上进行修改。因此，下面这些表述是等价的：

In [134]:
# 常见的非函数方式
df2 = df.copy()
v = df2['col1'] - df2['col1'].mean()
df2['k'] = v

# 函数赋值的方式
df2 = df.assign(k=v)

原位赋值可能比使用`assign`更为快速，但`assign`可以实现更方便的方法链：

In [138]:
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('key')
          .col1_demeaned.std())

使用外部的小括号来使添加换行符更方便。

在做方法链时要牢记可能会需要引用临时对象。在之前的例子中，无法引用load_data的结果，除非它被赋值给临时变量df。为了处理这种情况，`assign`和很多其他的pandas函数接受函数型的参数，这种参数也被称为可调用参数。

为了展示操作中的可调用对象，考虑下面这段之前讲过的代码段：
```
df = load_data()
df2 = df[df['col2'] < 0]
```
上面的代码可以改写为：
```
df = (load_data()
      [lambda x: x['col2'] < 0])
```
这里，load_data的结果没有复制给一个变量，因此传递进`[]`的函数将会被绑定到方法链那一阶段的对象上。

之后，可以继续将整个序列写作一个单链表达式：
```
result = (load_data()
          [lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          .groupby('key')
          .col1_demeaned.std())
```
无论是否倾向于按这种风格写代码都是偏好问题，但将表达式分解为多个步骤可能会使代码更具可读性。

### 12.3.1　pipe方法
使用内建的pandas函数和刚才看到的用可调用参数进行方法链接的方式，可以完成很多工作。然而，有时需要使用自定义的函数或来自第三方库的函数。这就是`pipe`（管道）方法出现的原因。

考虑下面一个函数调用序列：
```
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```
在使用接受并返回Series或DataFrame对象的函数时，可以调用`pipe`方法重写代码：
```
result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
```

表达式`f（df）`和`df.pipe（f）`是等价的，但是`pipe`使得链式调用更为方便。

将操作的序列泛化成可复用的函数是`pipe`方法的一个潜在用途。作为示例，考虑从一列中减去分组平均值：
```
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```
假设想要对多个列去除均值并方便地改变分组键。此外，可能想要将转换在方法链中执行。下面是一个示例实现：
```
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```
之后可以写如下代码：
```
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))
```